In [1]:
# !pip install yfinance

In [3]:
import yfinance as yf
import pandas as pd

In [4]:
stocks = pd.read_csv('selected stocks.csv') # list of stock tickers to download
stocks.head()

,Company,Ticker
0,Apple Inc.,AAPL
1,Archer Daniels Midland,ADM
2,Ametek,AME
3,A.O. Smith,AOS
4,Dennison Manufacturing,AVY


In [5]:
tickers = stocks['Ticker'].tolist()

# Download data

In [6]:
data = yf.download(tickers, interval="1d", start="1990-01-01")

[*********************100%%**********************]  81 of 81 completed


10 Failed downloads:
['NAV', 'REV', 'DFODQ', 'BGG']: Exception('%ticker%: No timezone found, symbol may be delisted')
['TSO', 'FCS', 'JOY', 'GKIS', 'BS', 'GMIL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1990-01-01 -> 2023-11-12)')


In [8]:
data.head()

Adj Close                                          \
                         AAPL       ADM       AME       AOS       AVY   
Date                                                                    
1990-01-02 00:00:00  0.263414  4.656210  1.292103  0.353147  7.014159   
1990-01-03 00:00:00  0.265181  4.656210  1.304178  0.349752  7.014159   
1990-01-04 00:00:00  0.266066  4.630906  1.292103  0.353147  7.094787   
1990-01-05 00:00:00  0.266950  4.428463  1.280027  0.349752  6.879792   
1990-01-08 00:00:00  0.268718  4.479074  1.267951  0.353147  6.879792   

                                                                   ... Volume  \
                            BA        BC       BDX BGG        BKR  ...    TSO   
Date                                                               ...          
1990-01-02 00:00:00  11.043631  7.969816  4.594040 NaN  10.116336  ...    NaN   
1990-01-03 00:00:00  11.312985  7.901111  4.548192 NaN   9.969729  ...    NaN   
1990-01-04 00:00:00  11.268093  7.901111  4.511514 NaN   9.725367  ...    NaN   
1990-01-05 00:00:00  11.110964  7.901111  4.502344 NaN   9.676490  ...    NaN   
1990-01-08 00:00:00  11.245651  7.832400  4.529851 NaN   9.774240  ...    NaN   

                                                                               \
                        TTC       TXN     VFC     VMC     VMI VYX     WDC WRC   
Date                                                                            
1990-01-02 00:00:00  622400   3766400  654192   69300   30400 NaN  523200 NaN   
1990-01-03 00:00:00  318400   8323200  744250   19500  815200 NaN  506600 NaN   
1990-01-04 00:00:00  648000  10654400  493618   23400   96000 NaN  618400 NaN   
1990-01-05 00:00:00  544000   6528000  720461   51000  263800 NaN  345200 NaN   
1990-01-08 00:00:00  320000   2875200  186062  495300    5200 NaN  178800 NaN   

                              
                         XOM  
Date                          
1990-01-02 00:00:00  5326000  
1990-01-03 00:00:00  4980400  
1990-01-04 00:00:00  6013200  
1990-01-05 00:00:00  3854800  
1990-01-08 00:00:00  4302000  

[5 rows x 486 columns]

# Drop missing data

In [9]:
data = data.dropna(axis=1)

# Round floating point numbers

In [10]:
df_rounded = data.round(3)

# Unpivot the data

In [11]:
frames = []
for c in df_rounded.columns.get_level_values(0).unique():
    d = df_rounded[c].reset_index()
    d = d.melt(id_vars=['Date'], var_name='Ticker', value_name=c)
    frames.append(d)

base = frames[0]
for f in frames[1:]:
    base = base.merge(f, on=['Date', 'Ticker'], how='inner')
print(base.head())

        Date Ticker  Adj Close  Close   High    Low   Open     Volume
0 1990-01-02   AAPL      0.263  0.333  0.335  0.312  0.315  183198400
1 1990-01-03   AAPL      0.265  0.335  0.339  0.335  0.339  207995200
2 1990-01-04   AAPL      0.266  0.336  0.346  0.333  0.342  221513600
3 1990-01-05   AAPL      0.267  0.337  0.342  0.330  0.337  123312000
4 1990-01-08   AAPL      0.269  0.339  0.339  0.330  0.335  101572800


# Save the file

In [14]:
# Save the original downloaded data
base.to_csv("original_downloaded_stocks_data.csv", index=False)